Написать приложение, которое собирает основные новости с сайта на выбор news.mail.ru, lenta.ru, yandex-новости. Для парсинга использовать XPath. Структура данных должна содержать:
название источника;
наименование новости;
ссылку на новость;
дата публикации.
Сложить собранные новости в БД

In [1]:
from lxml import html
from pprint import pprint
import requests
import datetime
import pandas as pd
header = {'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 YaBrowser/21.9.1.600 (beta) Yowser/2.5 Safari/537.36'}
pd.options.display.max_colwidth = 100

In [2]:
def lenta_news_agg():
    url = 'https://lenta.ru'
    response = requests.get(url, headers=header)
    dom = html.fromstring(response.text)
    articles = dom.xpath("//div[contains(@class, 'b-yellow-box__wrap')]/div/a")    
    lenta_news = []
    for article in articles:
        event = {}
        
        event['name'] = article.xpath('.//text()')[0].replace('\xa0', ' ')

        event['link'] = url + article.xpath('.//@href')[0]

        event['date'] = datetime.datetime.strptime('-'.join(article.xpath('.//@href')[0].split('/')[2:5]), '%Y-%m-%d').date()

        event['source'] = 'lenta.ru'
        lenta_news.append(event)
    return lenta_news


In [29]:
def mailru_news_agg():
    url = 'https://news.mail.ru/'
    response = requests.get(url, headers=header)
    dom = html.fromstring(response.text)
    links = []
    links.extend(dom.xpath("//table/tr/td[1]/div[@class='daynews__item daynews__item_big']/a/@href"))    
    links.extend(dom.xpath("//table/tr/td[2]/div[contains(@class, 'daynews__item')]/a/@href"))
    links.extend(dom.xpath("//div[@class='js-module']/ul/li[contains(@class,'list__item')]/a/@href"))
    mail_events = []
    for link in links:
        event = {}
        curr_link_response = requests.get(link, headers=header)
        curr_dom = html.fromstring(curr_link_response.text)
        event['link'] = link
        try:
            event['date'] = datetime.datetime.strptime(curr_dom.xpath(".//h1[@class='hdr__inner']/../../../../div[contains(@class, 'breadcrumbs')]/span/span/span[contains(@class, 'js-ago')]/@datetime")[0][0:10], '%Y-%m-%d').date()
            event['source'] = curr_dom.xpath(".//h1[@class='hdr__inner']/../../../../div[contains(@class, 'breadcrumbs')]/span/span/a/span/text()")[0]
            event['name'] = curr_dom.xpath(".//h1[@class='hdr__inner']/text()")[0]
        except Exception:
            continue
        mail_events.append(event)
    return mail_events
#                                                    

In [30]:
def to_db():
    def make_clickable(val):
        # target _blank to open new window
        return '<a target="_blank" href="{}">{}</a>'.format(val, val)
    df_temp = pd.DataFrame(lenta_news_agg() + mailru_news_agg(), columns=['name', 'date', 'link', 'source'])
    df_temp = df_temp.style.format({'link': make_clickable})
    return df_temp

In [31]:
to_db()

,name,date,link,source
0,Стали известны шансы на спасение оставшихся в шахте «Листвяжная» горняков,2021-11-25,https://lenta.ru/news/2021/11/25/gornyaki/,lenta.ru
1,Главу ФСИН Калашникова отправили в отставку,2021-11-25,https://lenta.ru/news/2021/11/25/fsinn/,lenta.ru
2,Госдума лишила депутата Рашкина неприкосновенности,2021-11-25,https://lenta.ru/news/2021/11/25/rashkin/,lenta.ru
3,Десятки людей пропали после пожара в шахте «Листвяжная» в Кузбассе,2021-11-25,https://lenta.ru/news/2021/11/25/kuzb/,lenta.ru
4,Названы главные киберугрозы будущего,2021-11-25,https://lenta.ru/extlink/2021/11/25/cyber_threat/,lenta.ru
5,Биолог объяснила опасность самоликвидации дельта-штамма коронавируса в России,2021-11-25,https://lenta.ru/news/2021/11/25/deltaaa/,lenta.ru
6,В России снизилось число бессимптомных носителей COVID-19,2021-11-25,https://lenta.ru/news/2021/11/25/nositely/,lenta.ru
7,Байден решил добиться исключения санкций против «Северного потока-2»,2021-11-25,https://lenta.ru/news/2021/11/25/bayden_sankcii/,lenta.ru
8,Самые опасные хакеры из России и стран бывшего СССР — кто они?,2021-11-22,https://lenta.ru/extlink/2021/11/22/hakeri/,lenta.ru
9,Выживших в шахте «Листвяжная» нет. Общее число погибших составило 52 человека,2021-11-25,https://news.mail.ru/incident/48948611/,ТАСС


In [127]:
# url = 'https://news.yandex.ru'
# response = requests.get(url, headers=header)
# pprint(response.text)
# dom = html.fromstring(response.text)
# links = dom.xpath("//a[contains(@class, 'mg-card__link')]/@href")
# for link in links:
#     print(link)

('<!doctype html><html prefix="og: http://ogp.me/ns#"><meta '
 'http-equiv="X-UA-Compatible" content="IE=edge"><meta charset="utf-8"><meta '
 'name="viewport" content="width=device-width,initial-scale=1"><title '
 'data-react-helmet="true">Ð\x9eÐ¹!</title><meta data-react-helmet="true" '
 'property="og:title" content="Ð¯Ð½Ð´ÐµÐºÑ\x81"><meta data-react-helmet="true" '
 'property="og:description" content="Ð\x9dÐ°Ð¹Ð´Ñ\x91Ñ\x82Ñ\x81Ñ\x8f '
 'Ð²Ñ\x81Ñ\x91"><meta data-react-helmet="true" property="og:image" '
 'content="https://yastatic.net/s3/home/logos/share/share-logo-ru.png"><link '
 'rel="stylesheet" href="/captcha_smart.min.css?k=1632998364813"><div '
 'id="root"><div class="Theme Theme_color_yandex-default '
 'Theme_root_default"><div class="Container"><div class="Spacer" '
 'style="padding-bottom:40px"><a href="https://www.yandex.ru" '
 'title="Ð¯Ð½Ð´ÐµÐºÑ\x81" class="Link Link_view_default LogoLink"><svg '
 'width="86" height="36" viewBox="0 0 86 36"><path d="M45.983 '
 '28.888H44.